In [ ]:
from fastapi import FastAPI, File, UploadFile
import shutil
from PIL import Image
import numpy as np
import tensorflow as tf
import uvicorn

app = FastAPI()

# Load your TensorFlow model
model = tf.keras.models.load_model(r"C:\Users\filip\Documents\GitHub\2024-25c-fai1-adsai-0FilipSarik243537\best_model.h5")  # Replace with your actual model file
model = tf.keras.model.load_weights(r"C:\Users\filip\Documents\GitHub\2024-25c-fai1-adsai-0FilipSarik243537\best_model_weights.h5")  # Replace with your actual model file

# Function to preprocess image
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = image.resize((224, 224))  # Adjust size based on your model input size
    image_array = np.array(image) / 255.0  # Normalize pixels
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    return image_array

# Function to make a prediction
def predict_image(image_path):
    processed_image = preprocess_image(image_path)
    
    # Get prediction
    predictions = model.predict(processed_image)
    predicted_class = np.argmax(predictions, axis=1)[0]  # Get the class label
    confidence = np.max(predictions)  # Get confidence score

    return predicted_class, confidence

# API Endpoint to Receive Image and Predict
@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    file_location = "temp_image.jpg"

    # Save uploaded image
    with open(file_location, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)

    # Get prediction
    prediction, confidence = predict_image(file_location)

    return {
        "prediction": str(prediction),
        "confidence": float(confidence)
    }

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)